# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [92]:
# import libraries
import re
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_pipeline.db')
df = pd.read_sql_table('disaster_messages', con=engine)
X_labels = ['id', 'message', 'original', 'genre']
X = df[X_labels]
Y = df.drop(columns=X_labels)

In [5]:
X.head(1)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct


In [6]:
Y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    
    token_list = word_tokenize(text)
    token_list = [token.strip() for token in token_list if token not in stopwords.words("english")]
    token_list = [lemmatizer.lemmatize(token) for token in token_list]
        
    return token_list

In [17]:
tokenize(X.message[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [40]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
#    ('classifier', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ('classifier', RandomForestClassifier(n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X['message'], Y)

In [42]:
X_test.shape

(6545,)

In [100]:
pipeline.fit(X_train, Y_train);

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
Y_pred = pipeline.predict(X_test)

In [99]:
classification_reports = {}
for i, column in enumerate(Y_test):
    classification_reports[column] = classification_report(Y_test.values[i], Y_pred[i])
    print('Label: {}'.format(column))
    print('-'*50)
    print(classification_reports[column])
    #print(precision_recall_fscore_support(Y_test.values[i], Y_pred[i]))

Label: related
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        31
           1       0.00      0.00      0.00         5

    accuracy                           0.86        36
   macro avg       0.43      0.50      0.46        36
weighted avg       0.74      0.86      0.80        36

Label: request
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.74      1.00      0.85        26
           1       1.00      0.10      0.18        10

    accuracy                           0.75        36
   macro avg       0.87      0.55      0.52        36
weighted avg       0.81      0.75      0.67        36

Label: offer
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        29
           1       1.00      0.14      0.

In [91]:
Y_test['medical_help'].shape - Y_test['medical_help'].sum()

array([6053])

In [68]:
Y_test.values[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [67]:
Y_pred[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### 6. Improve your model
Use grid search to find better parameters. 

In [103]:
parameters = {
    'classifier__n_estimators': [10, 50, 100],
    'classifier__min_samples_split': [2,4,6],
    'classifier__max_depth': [5,10,None],
    'classifier__max_features': [.1, .25, .5, 'sqrt', 'log2']
}

cv = GridSearchCV(pipeline, parameters, scoring='recall', n_jobs=-1, verbose=1)

In [104]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 135 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.